# Cubeviz base functionality with leaflet

1. Load a data cube
2. Scrub through the channels, displaying 2D and 1D
3. Median collapse cube
4. Stretch goal: Hover stats at cursor (x, y, counts, RA, Dec)

In [ ]:
import io
import logging
import random
import threading
import warnings

import numpy as np

from astropy import visualization
from astropy.nddata import CCDData
from skimage import transform

import ipyleaflet
import ipywidgets

from IPython.display import display

import flask

from matplotlib import pyplot as plt

Create sample data cube.

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS

with fits.open('/redkeep/ironthrone/ssb/stginga/test_data/ngc6946.fits') as pf:
    cube = pf[0].data
    hdr = pf[0].header
    wcs = WCS(hdr)
    ylbl_1d = hdr['BUNIT']
    xlbl_1d = hdr['CTYPE3']
    flux_unit = 'Jy'
    
# NOTE: Random cube if you cannot access the data file
#np.random.seed(1234)
#cube = np.random.random(10000000).reshape((1000, 100, 100))
#wcs = None
#ylbl_1d = 'count'
#xlbl_1d = 'index'
#flux_unit = ylbl_1d

In [ ]:
c1 = CCDData(cube, unit=flux_unit, wcs=wcs)

In [ ]:
c1.wcs

Some functions copied over from `html_view_into_leaflet.ipynb`.

**TODO: These functions should be documented because I have no idea what they are supposed to do.**

In [ ]:
"""
Writing PNGs
"""

#-----------------------------------------------------------------------------
# Copyright (c) 2013, yt Development Team.
#
# Distributed under the terms of the Modified BSD License.
#
# The full license is in the file COPYING.txt, distributed with this software.
#-----------------------------------------------------------------------------

import matplotlib._png as _png

from io import BytesIO as StringIO


def call_png_write_png(buffer, width, height, filename, dpi):
    _png.write_png(buffer, filename, dpi)

    
def write_png(buffer, filename, dpi=100):
    width = buffer.shape[1]
    height = buffer.shape[0]
    call_png_write_png(buffer, width, height, filename, dpi)

    
def write_png_to_string(buffer, dpi=100, gray=0):
    width = buffer.shape[1]
    height = buffer.shape[0]
    fileobj = StringIO()
    call_png_write_png(buffer, width, height, fileobj, dpi)
    png_str = fileobj.getvalue()
    fileobj.close()
    return png_str

In [ ]:
# SERVER:

visdat = None


# NOTE: Unlike html_view_into_leaflet.ipynb, there is extra
# arg named i_slice here for slicing the cube.
def re_stretch(stretch, i_slice):
    global visdat
    visdat = np.flip(stretch(c1.data[i_slice]), 0)


def ccd_to_pngstr_app(dat):
    return write_png_to_string((dat*255).astype('uint8')[:,:,np.newaxis])


logstream = io.StringIO()
logging.basicConfig(stream=logstream)

app = flask.Flask(__name__)


# https://en.wikipedia.org/wiki/Tiled_web_map
# Taken from Leaflet_image_viewer.ipynb
@app.route('/fits<int:cachebuster>/<string:z>/<int:x>/<int:y>.png')
def get_subfits(z, x, y, cachebuster):
    # Zoom
    z = int(z)
    
    if z < 0:  # TODO: Not used?
        factor = 2 ** -z
        wid = 256 * factor
        xrng = slice(x * wid, (x + 1) * wid, factor)
        yrng = slice(y * wid, (y + 1) * wid, factor)

        subdat = visdat[yrng, xrng]
        
    else:
        if z == 1:  # Default zoom
            wid = 256
        else:    
            wid = 256 // (2 ** z)
            
        xrng = slice(x * wid, (x + 1) * wid)
        yrng = slice(y * wid, (y + 1) * wid)
        subdat = visdat[yrng, xrng]
        
        if z > 1 or (z == 1 and subdat.shape[0] < 256 and subdat.shape[1] < 256):
            # TODO: Did not work for upper-right and lower-left quadrants.
            #subdat = subdat.repeat(z, 0).repeat(z, 1)
            # TODO: Prettier but still buggy.
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                subdat = transform.resize(subdat, (256, 256), mode='constant')
            wid = subdat.shape[0]
            
    if subdat.shape != (wid, wid):
        if 0 in subdat.shape:
            1 / 0  # TODO: Why not raise exception explicitly?
        else:
            # Pad out with NaNs
            temp = subdat.copy()
            subdat = np.empty((256, 256), dtype=temp.dtype)
            subdat.fill(np.nan)
            subdat[:temp.shape[0], :temp.shape[1]] = temp
            
    return ccd_to_pngstr_app(subdat)

Set your favorite display stretch and cuts.

In [ ]:
fav_stretch = visualization.LogStretch() + visualization.PercentileInterval(95)

**TODO: Do users need to worry about server warning from `th.start()`?**

In [ ]:
th = threading.Thread(target=lambda:app.run(debug=False, use_reloader=False, port=5013))
th.start()

Create an empty `ipyleaflet` viewer. We will display here later.

**TODO: It looks like ipyleaflet is solely to support Earth map view, so using it like this is too hacky?**

**TODO: Theoretically, a subset of this code can be re-purposed to display median-collapsed cube in the same way.**

In [ ]:
url_templ = 'http://127.0.0.1:5013/fitsNUM/{z}/{x}/{y}.png'

m = ipyleaflet.Map(crs='Simple', center=(0, 0), zoom=1, min_zoom=1,
                   layers=[], scroll_wheel_zoom=True)

# NOTE: Need to put this here for 1D plotting to update.
# NOTE: In-place plot update does not work with inline.
# TODO: Does not work in Jupyter Lab
%matplotlib notebook

# For 1D plotting
fig, ax = plt.subplots()

# https://github.com/jupyter-widgets/ipyleaflet/issues/332
lbl = ipywidgets.Label()
display(lbl)

# For debugging
lbl2 = ipywidgets.Label()
display(lbl2)

# Event names taken from
# https://github.com/jupyter-widgets/ipyleaflet/blob/5f27207ac7a3f2a08f45181613e9ed9ab37eb759/ipyleaflet/leaflet.py#L100
def handle_interaction(**kwargs):
    global lbl, lbl2
    event_type = kwargs.get('type')
    coo = kwargs.get('coordinates')
    disp_size = 128
    
    lbl2.value = 'LEAF_X: {:.2f}, LEAF_Y: {:.2f}, Zoom: {}'.format(coo[1], coo[0], m.zoom)
    
    if visdat is None:
        return
    
    # Rough map coord to X, Y translation
    img_y = (coo[0] + disp_size) * visdat.shape[0] / disp_size
    img_x = coo[1] * visdat.shape[1] / disp_size
    
    iy = int(img_y)
    ix = int(img_x)
    
    # TODO: Can we make this more robust?
    # See https://leafletjs.com/examples/crs-simple/crs-simple.html
    if event_type == 'mousemove':       
        if (0 <= ix < visdat.shape[1] and
                0 <= iy < visdat.shape[0]):
            img_val = cube[slider_obj.value, iy, ix]
        else:
            img_val = np.nan
            
        if c1.wcs is None:
            radec_str = ''
        else:
            skycoord = c1.wcs.pixel_to_world(img_x, img_y, slider_obj.value)[0]
            # TODO: Format RA and Dec to nicer output
            radec_str = ' (RA: {}, Dec: {})'.format(str(skycoord.ra), str(skycoord.dec))
            
        lbl.value = 'Value: {:.4e}, X: {:.2f}, Y: {:.2f} '.format(
            img_val, img_x, img_y) + radec_str
        
    elif event_type == 'click':
        ax.clear()
        
        # Matplotlib for 1D
        if (0 <= ix < visdat.shape[1] and
                0 <= iy < visdat.shape[0]):
            data_1d = cube[:, iy, ix]
            ax.plot(data_1d)
            ax.set_ylabel(ylbl_1d)
            ax.set_xlabel(xlbl_1d)
            fig.canvas.draw_idle()


# NOTE: Random int here is how they do cache invalidation...
cachebuster_int = random.randint(0, 1000000)
local_fits_layer = ipyleaflet.basemap_to_tiles(
    {'url': url_templ.replace('NUM', str(cachebuster_int)), 
     'attribution': 'fitsfile'})
local_fits_layer.cachebuster_int = cachebuster_int


def refresh():
    local_fits_layer.cachebuster_int += 1
    local_fits_layer.url = url_templ.replace('NUM', str(local_fits_layer.cachebuster_int))
    
    
def rere_stretch(stretch, i_slice):
    re_stretch(stretch, i_slice)
    refresh()

    
def change_slice(x):
    rere_stretch(fav_stretch, x)

    
m.add_layer(local_fits_layer)
m.on_interaction(handle_interaction)

m

The following hooks up the slider to display chosen slice in the cube on the viewer above.

In [ ]:
slider_obj = ipywidgets.IntSlider(min=0, max=cube.shape[0]-1, step=1, value=0)
interact_obj = ipywidgets.interact(change_slice, x=slider_obj)

End of prototype demo.